In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

## The data

In [2]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1532,7.5,0.230,0.49,7.7,0.049,61.0,209.0,0.99410,3.14,0.30,11.1,7
1721,8.2,0.150,0.48,2.7,0.052,24.0,190.0,0.99500,3.50,0.45,10.9,7
3658,5.8,0.275,0.30,5.4,0.043,41.0,149.0,0.99260,3.33,0.42,10.8,7
959,7.3,0.240,0.34,15.4,0.050,38.0,174.0,0.99830,3.03,0.42,9.0,6
1592,6.8,0.210,0.74,1.2,0.047,25.0,111.0,0.99160,3.13,0.41,10.7,6
3137,6.2,0.250,0.39,1.3,0.051,42.0,135.0,0.99060,3.23,0.40,11.1,6
4232,6.6,0.390,0.28,9.2,0.036,10.0,92.0,0.99206,3.07,0.35,12.1,6
819,6.6,0.560,0.22,8.9,0.034,27.0,133.0,0.99675,3.20,0.51,9.1,5
2430,6.4,0.280,0.17,8.3,0.042,61.0,195.0,0.99577,3.22,0.46,9.4,5
4632,6.3,0.300,0.91,8.2,0.034,50.0,199.0,0.99394,3.39,0.49,11.7,6


## 1. Tracking experiments

### Tracking stores
MLflow supports two types of backend stores: *file store* and *database-backed* store.

- Local file path (specified as file:/my/local/dir), where data is just directly stored locally. Defaults to `mlruns/`
- Database encoded as <dialect>+<driver>://<username>:<password>@<host>:<port>/<database>. Mlflow supports the dialects mysql, mssql, sqlite, and postgresql. For more details, see SQLAlchemy database uri.
- HTTP server (specified as https://my-server:5000), which is a server hosting an MLFlow tracking server.
- Databricks workspace (specified as databricks or as databricks://<profileName>, a Databricks CLI profile.
    
### Artifact stores
- Amazon S3
- Azure Blob Storage
- Google Cloud Storage
- FTP server
- SFTP Server
- NFS
- HDFS

Start the MLflow tracking server by 

```
mlflow server \
    --backend-store-uri /mnt/persistent-disk \
    --default-artifact-root s3://my-mlflow-bucket/ \
    --host 0.0.0.0
    --port 5000
```

or use the default storage method to write to `mlruns/`.

In [4]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [5]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [6]:
exp_name = "ElasticNet_model_wine_dataset" #set an exp.
mlflow.set_experiment(exp_name)

INFO: 'ElasticNet_model_wine_dataset' does not exist. Creating a new experiment


### What do we track?

- **Code Version**: Git commit hash used for the run (if it was run from an MLflow Project)
- **Start & End Time**: Start and end time of the run
- **Source**: what code run?
- **Parameters**: Key-value input parameters.
- **Metrics**: Key-value metrics, where the value is numeric (can be updated over the run)
- **Artifacts**: Output files in any format.

In [7]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [8]:
# could also do
# with mlflow.start_run():
#     for epoch in range(0, 3):
#          mlflow.log_metric(key="quality", value=2*epoch, step=epoch)

In [9]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689
Save to: mlruns/4/d9c4acf165ad4bab89c2f2b983957960/artifacts


In [10]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093893
  R2: 0.20224631822892103
Save to: mlruns/4/8710b63278a64d4d888d8e71bd1f228b/artifacts


In [11]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlruns/4/21b6b967818c4755b5d221b055c04bd9/artifacts


### 1.1 Comparing runs
Run `mlflow ui` in a terminal or `http://your-tracking-server-host:5000` to view the experiment log and visualize and compare different runs and experiments. The logs and the model artifacts are saved in the `mlruns` directory (or where you specified).

## 2. Packaging the experiment as a MLflow project as conda env

Specify the entrypoint for this project by creating a `MLproject` file and adding an conda environment with a `conda.yaml`. You can copy the yaml file from the experiment logs.

To run this project, invoke `mlflow run . --no-conda -P alpha=0.7`. After running this command, MLflow runs your training code in a new Conda environment with the dependencies specified in `conda.yaml`.


## 3. Deploy the model

Deploy the model locally by running 

`mlflow models serve -m mlruns/0/f5f7c052ddc5469a852aa52c14cabdf1/artifacts/model/ --no-conda -h 0.0.0.0 -p 1234`

Test the endpoint:

`curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://0.0.0.0:1234/invocations`

You can also simply build a docker image from your model

`mlflow models build-docker -m mlruns/1/d671f37a9c7f478989e67eb4ff4d1dac/artifacts/model/ -n elastic_net_wine`

and run the container with

`docker run -p 8080:8080 elastic_net_wine`.

Or you can directly deploy to AWS sagemaker or Microsoft Azure ML.

## 4. Tagging runs

In [12]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>, <Experiment: artifact_location='mlruns/4', experiment_id='4', lifecycle_stage='active', name='ElasticNet_model_wine_dataset', tags={}>, <Experiment: artifact_location='mlruns/3', experiment_id='3', lifecycle_stage='active', name='ElasticNet_wine_data', tags={}>, <Experiment: artifact_location='mlruns/2', experiment_id='2', lifecycle_stage='active', name='ElasticNet_wine_dataset', tags={}>]


In [13]:
# get the run
_run = client.get_run(run_id="8710b63278a64d4d888d8e71bd1f228b")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6155290394093893,
 'r2': 0.20224631822892103,
 'rmse': 0.7859129997062341}, params={'alpha': '0.2', 'l1_ratio': '0.2'}, tags={'mlflow.log-model.history': '[{"run_id": "8710b63278a64d4d888d8e71bd1f228b", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2022-06-27 09:53:17.531403", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"loader_module": "mlflow.sklearn", '
                             '"python_version": "3.8.10", "env": '
                             '"conda.yaml"}, "sklearn": {"pickled_model": '
                             '"model.pkl", "sklearn_version": "0.22.1", '
                             '"serialization_format": "cloudpickle"}}}]',
 'mlflow.source.name': '/usr/lib/python3/dist-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'renuka'}>, info=<RunInfo: artifact_uri=

In [15]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)